## Train model and save it

This model predicts remission likelihood of breast cancer using the UCI breast cancer dataset

In [37]:
%run train.py

X shape: :  (286, 43)
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)
Accuracy: 68.42%


## Import flighty SDK and initialize

In [26]:
import flighty as f

# f.set_hostname('127.0.0.1:8002')

In [36]:
f.create_endpoint(name='doc-rec')
f.create_artifact(path='./model_dir', name='xgboost', version=0)
f.create_handler(endpoint='doc-rec', name='xgboost', 
  docker_image='docker.io/gvashishtha/flighty:flighty-demo',
  code_path='./code_sample', artifacts=[{'name': 'xgboost', 'version': 0}])

{'name': 'doc-rec', 'url': 'http://127.0.0.1/doc-rec'}
{'name': 'xgboost',
 'path': '/code/flighty-files/xgboost/0',
 'type': 'model',
 'version': 0}
{'name': 'doc-rec-xgboost-0-code',
 'path': '/code/flighty-files/doc-rec-xgboost-0-code/0',
 'type': 'code',
 'version': 0}
{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
 'endpoint': 'doc-rec',
 'name': 'xgboost',
 'version': 0}


In [31]:
response = f.invoke(endpoint='doc-rec', body={"data": {
    "age": "40-49",
    "meno": "lt40",
    "tumor_size": "0-4",
    "inv_nodes": "0-2",
    "node_caps": "yes",
    "deg_malig": "1",
    "breast": "left",
    "breastquad": "left_up",
    "irradiat": "yes"
  }})
print(response.text, response.status_code)

"Your prediction is ['no-recurrence-events']" 200


In [32]:
f.list_handlers()

[{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
  'endpoint_id': 1,
  'id': 1,
  'name': 'xgboost',
  'prod_traffic': 100,
  'shadow_traffic': 0,
  'version': 0}]


## Clean up

In [38]:
f.delete_handler(endpoint='doc-rec', name='xgboost', version=0)
f.delete_endpoint(name='doc-rec')
f.delete_artifact(name='xgboost', version=0)

None
'Artifact doc-rec-xgboost-0-code with version 0 successfully deleted.'
'Endpoint doc-rec successfully deleted.'
'Artifact xgboost with version 0 successfully deleted.'


## Confirm our resources were deleted

In [39]:
f.list_artifacts()
f.list_endpoints()
f.list_handlers()

[]
[]
[]
